In [43]:
import os
import time
from gtts import gTTS
import speech_recognition as sr
from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from db_search import get_info as search

In [44]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    api_key="I5uzjp4ZXUioIEM7hdYgdJtpv4NUlWov"
)

In [37]:
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma(persist_directory="./vector_db", embedding_function=embedder)

def get_response(user_query, doc_context):
    llm_query_res_template = """
        Answer the question based on the context below. If the question cannot be answered using the information provided, reply with "I don't know". Also, make sure to answer the following questions considering the history of the conversation:
        You are an intelligent virtual financial assistant for Predixion AI, directly engaging with customers about their loan repayments. Your role is to help manage their loan, facilitate payments, and answer financial questions in a clear, professional way. Communicate in a friendly, authoritative manner, addressing the customer directly ("you") with concise responses suitable for WhatsApp.
        Make sure you communicate with the user in such a way that your response should always lead to payment collection.
        Based on the user question, you should respond in a short way. Do not write much; it should be short and precise.

        Instructions:
        1. Use precise financial language and ensure clear, accurate information.
        2. Facilitate payments: If the user is willing to pay the loan then please provide this link '''https://paymentUSER1UDN.com'''. Do not send the link until user requests or user wants to pay the loan.
        3. Offer solutions: If the customer is struggling, provide options like grace periods, payment restructuring, or deadline extensions.
        4. Keep responses short and to the point.
        5. Ensure confidentiality and remind the customer to keep their payment details secure.
        6. You can only extend the loan dates by 10 days if user requests for grace periods or deadline extensions.

        Context: "NONE"
        Question: {user_query}
        Doc context: {doc_context}
        Answer:
    """

    prompt_query_res_template = ChatPromptTemplate.from_template(llm_query_res_template)
    llm_chain = prompt_query_res_template | llm | StrOutputParser()

    '''response = llm_chain.stream({
        "user_query": user_query,
        "context": context,
        "doc_context": doc_context,
    })'''

    response = ''.join([chunk for chunk in llm_chain.stream({
        "user_query": user_query,
        #"context": context,
        "doc_context": doc_context,
    })])

    return response

In [39]:
def get_info(phone):
    if phone in dataset['Mobile_No'].values:
        return dataset[dataset['Mobile_No'] == phone][data]
    else:
        return None

In [42]:
person = get_info(9988953565)
person

,F_Name,L_Name,Gender,Income,Bureau_score,Loan_amount,Loan_type,Interest_Rate,Interest,Loan_Processing_Fee,Current_balance,Installment_Amount,Disbursal_Date,Repayment_Start_Date,Repayment_tenure,Date_of_last_payment,Repayment_mode,No_of_late_payments
0,Myra,Das,Female,411116.9,708,43431.0,Consumer Durable Loan,10.8,2345.3,434.3,23371.7,7468.2,2023-09-02,2023-10-02,6,2024-07-14,UPI,0


In [1]:
type(9988953565)

int